In [ ]:
# This the Part 2 of this homework
%pip install transformers
%pip install torch
%pip install datasets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Load transformers pipeline - Part 1
from transformers import pipeline

# Built in sentiment analysis pipeline - Part 1
#model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
classifier = pipeline("sentiment-analysis", model=model_name)
results = classifier(["I love you", "I hate you"])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 9)}")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


label: POS, with score: 0.991669536
label: NEG, with score: 0.980660021


In [15]:
import torch
torch.cuda.is_available()

False

In [16]:
from datasets import load_dataset
imdb = load_dataset("imdb")

In [17]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [19]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

In [20]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # speed up training with padding

In [21]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    metric_accuracy = evaluate.load("accuracy")
    metric_f1 = evaluate.load("f1")
  
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = metric_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [23]:
%pip install ipywidgets
%pip install transformers
%pip install transformers[torch]
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: no matches found: transformers[torch]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-3000-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/var/folders/p_/l4md4zh5543c74k3yln2x1t80000gn/T/ipykernel_36755/1541456166.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [27]:
trainer.train()

  0%|          | 0/376 [00:00<?, ?it/s]

TrainOutput(global_step=376, training_loss=0.2960815632596929, metrics={'train_runtime': 662.781, 'train_samples_per_second': 9.053, 'train_steps_per_second': 0.567, 'total_flos': 782725021021056.0, 'train_loss': 0.2960815632596929, 'epoch': 2.0})

In [ ]:
%pip install scikit-learn

trainer.evaluate()

{'eval_loss': 0.31869709491729736,
 'eval_accuracy': 0.8666666666666667,
 'eval_f1': 0.8666666666666667,
 'eval_runtime': 41.2403,
 'eval_samples_per_second': 7.274,
 'eval_steps_per_second': 0.461,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/arpinkus/finetuning-sentiment-model-3000-samples/commit/aa700b4c6b3711730eb94626b1284966131e4dfe', commit_message='End of training', commit_description='', oid='aa700b4c6b3711730eb94626b1284966131e4dfe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/arpinkus/finetuning-sentiment-model-3000-samples', endpoint='https://huggingface.co', repo_type='model', repo_id='arpinkus/finetuning-sentiment-model-3000-samples'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline
 
sentiment_model = pipeline(model="federicopascual/finetuning-sentiment-model-3000-samples")
sentiment_model(["I love this move", "This movie sucks!"])

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.9558863043785095},
 {'label': 'LABEL_0', 'score': 0.9413503408432007}]